#Qiskit

In [ ]:
# Install Qiskit

!pip install qiskit
!pip install qiskit-ibm-runtime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 30.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
    

In [ ]:
# Import packages to build circuits

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter, ParameterVector
from qiskit.quantum_info import SparsePauliOp
import numpy as np

service = QiskitRuntimeService()

num_qubit = 30
depth = 3
theta = Parameter('θ')
phis = ParameterVector('φ', num_qubit)
qc = QuantumCircuit(num_qubit)

# Initialize a state
for q in range(0, num_qubit, 2):
    qc.x(q)

# Build dynamics
for d in range(depth):
    # Even layers
    for even_qubit in range(0, num_qubit, 2):
        qc.cz(even_qubit, even_qubit+1)
        qc.u(theta, 0, -np.pi, even_qubit)
        qc.u(theta, 0, -np.pi, even_qubit+1)
    # Local random phase gates
    for q in range(num_qubit):
        qc.p(phis[q], q)
    # Odd layers
    for odd_qubit in range(1, num_qubit-1, 2):
        qc.cz(odd_qubit, odd_qubit+1)
        qc.u(theta, 0, -np.pi, odd_qubit)
        qc.u(theta, 0, -np.pi, odd_qubit+1)
    # Local random phase gates
    for q in range(num_qubit):
        qc.p(phis[q], q)

# Define measurement observable
obs = SparsePauliOp('I' + 'Z' + 'I' * (num_qubit-2))

# Specify circuit parameter values
np.random.seed(0) # Specify the seed for debugging purpose such that the circuit is the same very time we run it
phi_max = 0.5 * np.pi
parameter_values = [np.random.uniform(-1 * phi_max, phi_max) for _ in range(num_qubit)] + [0.01 * np.pi]

AccountNotFoundError: 'Unable to find account.'

In [ ]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

backend = service.backend('ibm_nazca')
target = backend.target

# Transpile the circuit
pm = generate_preset_pass_manager(
        target=target,
        optimization_level=3
     )
t_qc = pm.run(qc)

# Map the observable according to the transpile layout
t_obs = obs.apply_layout(t_qc.layout)

NameError: name 'service' is not defined

In [ ]:
from qiskit_aer.primitives import Estimator

# To get a sense of good and bad, simulate the circuit to get the ideal, noiseless result

estimator = Estimator(skip_transpilation=True)

sim_job = estimator.run(
              t_qc,
              t_obs,
              parameter_values,
              shots=10000
          )

sim_result = sim_job.result()

ModuleNotFoundError: No module named 'qiskit_aer'

In [ ]:
from qiskit_ibm_runtime import Options

options = Options()

# For comparison, we first run circuit with non-optimized parameters
options.transpilation.skip_transpilation = True
options.resilience_level = 0
options.execution.shots = 10000

estimator = Estimator(backend='ibm_nazca', options=options)
unoptimized_job = estimator.run(t_qc, t_obs, parameter_values)

# Get unoptimized result and plot
unoptimized_result = unoptimized_job.result()
unoptimized_result.values

# Next we redefine Runtime options with optimized parameters for transpilation and error mitigation
options = Options()

options.transpilation.skip_transpilation = True
options.resilience_level = 2
options.execution.shots = 10000

estimator = Estimator(backend='ibm_nazca', options=options)
optimized_job = estimator.run(t_qc, t_obs, parameter_values)

# Get optimized result and plot
optimized_result = optimized_job.result()
optimized_result.values

NameError: name 'Estimator' is not defined